In [11]:
!pip install transformers
!pip install einops
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 KB 6.3 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.0 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 KB 7.1 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 KB 7.3 MB/s eta 0:00:000:00:01m eta 0:

In [7]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, TextStreamer

In [8]:
torch.random.seed = 0

model_id = "tiiuae/falcon-7b-instruct" # "tiiuae/falcon-7b" #"tiiuae/falcon-40b-instruct" #"tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")

streamer = TextStreamer(tokenizer=tokenizer)

generator = pipeline("text-generation",
                    model=model_id,
                    tokenizer=tokenizer,
                    torch_dtype=torch.bfloat16,
                    trust_remote_code=True,
                    device=0,
                    max_length=500,
                    do_sample=True,
                    top_k=35,
                    top_p=.89,
                    temperature=.65,
                    num_return_sequences=1,
                    streamer=streamer,
                    pad_token_id = tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

In [12]:
import importlib

from generative_agent import agent
from generative_agent.agent import Agent, Dialogue, Action, Memory

import importlib
importlib.reload(agent)

<module 'generative_agent.agent' from '/home/alex/Programming/generative-agent-sim/generative_agent/agent.py'>

In [13]:
from pprint import pprint as pp

In [14]:
john_lin = Agent("John Lin", """John Lin is a pharmacy shopkeeper at the Willow Market and Pharmacy who loves to help people. 
He is always looking for ways to make the process of getting medication easier for his customers; John Lin
is living with his wife, Mei Lin, who is a college professor, and son, Eddy Lin, who is a student studying 
music theory; John Lin loves his family very much;""", "John is back home early from work.")
    
eddy_lin = Agent("Eddy Lin", """Eddy Lyn is a student studying music theory. He is passionated about music an loves to do walks in the 
nature. Eddy lives together with his mother Mei Lin and his Father John Lin. He is a very friendly person
and is always willing to help his friends.""", "Eddy is taking a short walk around his workplace")

In [24]:
relevant_memory = john_lin.memory.memory_for(eddy_lin)
john_observation = "John saw Eddy taking a short walk around his workplace."

john_action = Action(tokenizer, generator).generate(john_lin, john_observation, relevant_memory)
print(john_action[0])

John Lin is a pharmacy shopkeeper at the Willow Market and Pharmacy who loves to help people. 
He is always looking for ways to make the process of getting medication easier for his customers; John Lin
is living with his wife, Mei Lin, who is a college professor, and son, Eddy Lin, who is a student studying 
music theory; John Lin loves his family very much;

It is June 25, 2023, 10:57PM.
John Lin's status: John is back home early from work.

Observation: John saw Eddy taking a short walk around his workplace.

Summary of relevant context from John Lin's memory:
John Lin is Eddy Lin’s father. John Lin is caring and is interested to learn more about Eddy 
Lin’s school work. John Lin knows that Eddy Lin is working on a music composition.

Should John Lin react to the observation, and if so, what would be an appropriate reaction?
An appropriate reaction would be to smile at his son, Eddy Lin, and ask him how his day went. John Lin is also curious to know about his son’s school work, so he

In [34]:
dialogue = Dialogue(tokenizer, generator)

dialogue_history = []

dialogue_history.append(dialogue.start(john_lin, eddy_lin, john_observation, relevant_memory, john_action[0]))



John Lin: Hey, how was your day today?

Eddy Lin: Hey, it was a pretty good day. I finished my composition early, so I decided to take a short walk around the workplace.

John Lin: That’s great! I’m glad to hear that you finished your composition early. Did you have any other plans for


In [35]:
print(dialogue_history[-1][0])

John Lin: "Hey, how was your day today?"


In [36]:
observation = "John is in a conversation with Eddy"
relevant_memory = john_lin.memory.memory_for(eddy_lin)

dialogue_history.append(dialogue.turn(john_lin, eddy_lin, observation, relevant_memory, [turn for turn,_,_ in dialogue_history]))


John Lin: "It was a good day. I finished my work early. I'm glad to be home early. How was your day?"

What would Eddy Lin say to John Lin?
Eddy Lin: "I'm glad to hear that you finished your work early. I'm working on a music composition. I'


In [37]:
print(dialogue_history[-1][0])

John Lin: "It was a good day. I finished my work early. I'm glad to be home early. How was your day?"


In [38]:
observation = "Eddy is in a conversation with John"
relevant_memory = eddy_lin.memory.memory_for(john_lin)

dialogue_history.append(dialogue.turn(eddy_lin, john_lin, observation, relevant_memory, [turn for turn,_,_ in dialogue_history]))


Eddy Lin: "It was a good day too. I finished my work early and I went for a walk in the nature. It was a nice walk. I enjoyed it."

What would John Lin say to Eddy Lin?
John Lin: "That's great to hear. I'm glad that you enjoyed your walk. Do


In [39]:
for turn,_,_ in dialogue_history:
    print(turn)

John Lin: "Hey, how was your day today?"
John Lin: "It was a good day. I finished my work early. I'm glad to be home early. How was your day?"
Eddy Lin: "It was a good day too. I finished my work early and I went for a walk in the nature. It was a nice walk. I enjoyed it."


In [22]:
h = '\n'.join(dialogue_history[:5])  

action_prompt = f"""
Given the following dialogue:
{h}

Should John <continue> or <leave> the conversation?"""
        
with torch.no_grad():
    action = generator(action_prompt, 
                    max_length=len(tokenizer.encode(action_prompt)) + 30,
                    do_sample=True,
                    top_k=0,
                    top_p=.80,
                    temperature=.7,
                    streamer = None,
                    pad_token_id = tokenizer.eos_token_id
                    )
    
print(action[0]['generated_text'])

TypeError: sequence item 0: expected str instance, tuple found